# MODNet on experimental dataset 

MODNet model on the experimental dataset alone. This model is used as our benchmark in this Ensemble Methods repository.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import os
import copy

from collections import defaultdict
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt 
from IPython.display import Markdown
from matminer.datasets import load_dataset, get_all_dataset_info
from pymatgen.core import Composition

from modnet.featurizers import MODFeaturizer
from modnet.featurizers.presets import DeBreuck2020Featurizer

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

In [5]:
md = MODData.load('exp_gap_all')
df = pd.read_pickle('df_exp_cleaned.pkl')
md_joint = MODData.load('exp_pbe_joint')

2021-06-14 17:43:55,625 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f63c4940bb0> object, created with modnet version <=0.1.7
2021-06-14 17:43:56,239 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f6355795700> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_targets

,exp_gap,pbe_gap,difference
mp-12699,0.00,0.0000,0.0000
mp-559459,3.40,2.1704,1.2296
mp-21162,0.00,0.0000,0.0000
mp-1306,0.00,0.0000,0.0000
mp-15252,0.53,0.1049,0.4251
...,...,...,...
mp-556541,1.80,1.5135,0.2865
mp-2602,1.04,0.0000,1.0400
mp-30366,0.00,0.0000,0.0000
mp-3718,1.45,0.2418,1.2082


In [7]:
md_joint.df_targets = md_joint.df_targets.drop(columns=['pbe_gap', 'difference'])

In [8]:
md_joint.df_targets

,exp_gap
mp-12699,0.00
mp-559459,3.40
mp-21162,0.00
mp-1306,0.00
mp-15252,0.53
...,...
mp-556541,1.80
mp-2602,1.04
mp-30366,0.00
mp-3718,1.45


In [9]:
md_joint.df_structure

,structure
id,
mp-12699,"[[ 0.13616363 -2.60631371 1.50340165] Ca, [-1..."
mp-559459,"[[2.14049282 6.43464803 1.76199846] Cs, [-0.70..."
mp-21162,"[[0. 0. 0.] Sr, [0. 2.5259875 2.5259875..."
mp-1306,"[[3.259129 1.88166243 1.15971025] Th, [ 3.25..."
mp-15252,"[[0.95531657 1.042239 0.86229681] Cu, [2.803..."
...,...
mp-556541,"[[0. 6.49363948 5.82657217] In, [ 0. ..."
mp-2602,"[[3.023318 3.023318 3.023318] Nd, [0. 0. 0.] As]"
mp-30366,"[[4.51774558 1.37529211 4.32659015] Ca, [1.109..."


In [10]:
import uncertainty_toolbox as uct

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
uncertainties = np.ones(5)
metricss = []
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
      
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    ga = FitGenetic(train)
    model = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    metrics = uct.metrics.get_all_metrics(pred['exp_gap'].values, std['exp_gap'].values, true['exp_gap'].values)
    error = pred['exp_gap']-true['exp_gap']
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    std = std['exp_gap'].drop(std.index[((std['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    uncertainty = np.abs(std).mean()
    print('MAE =')
    print(mae)
    print('uncertainty =')
    print(uncertainty)
    maes[i] = mae
    uncertainties[i] = uncertainty
    metricss.append(metrics)

2021-06-14 17:43:56,483 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:56,503 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:56,515 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:56,534 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:56,545 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:56,560 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:56,569 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:56,584 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:56,593 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:56,607 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:57,411 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f6342db1ca0> object, created with modnet version 0.1.9
2021-06-14 17

100%|██████████| 100/100 [12:09<00:00,  7.30s/it]

2021-06-14 17:56:08,841 - modnet - INFO - Loss per individual: ind 0: 0.598 	ind 1: 0.654 	ind 2: 0.726 	ind 3: 0.610 	ind 4: 0.947 	ind 5: 0.768 	ind 6: 0.739 	ind 7: 0.736 	ind 8: 0.626 	ind 9: 0.651 	ind 10: 0.684 	ind 11: 0.741 	ind 12: 0.603 	ind 13: 0.589 	ind 14: 0.663 	ind 15: 0.635 	ind 16: 0.633 	ind 17: 0.682 	ind 18: 0.640 	ind 19: 0.735 	


2021-06-14 17:56:10,588 - modnet - INFO - Generation number 1
2021-06-14 17:56:13,295 - modnet - INFO - Multiprocessing on 10 cores. Total of 128 cores available.


100%|██████████| 100/100 [12:38<00:00,  7.59s/it]

...

 (3/n) Calculating adversarial group calibration metrics
  [1/2] for mean absolute calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


 10%|█         | 1/10 [00:00<00:01,  5.77it/s]

  [2/2] for root mean squared calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


100%|██████████| 10/10 [00:01<00:00,  5.28it/s]

 (4/n) Calculating sharpness metrics
 (n/n) Calculating proper scoring rule metrics
**Finished Calculating All Metrics**


===================== Accuracy Metrics =====================
  MAE           0.368
  RMSE          0.744
  MDAE          0.038
  MARPD         137.341
  R2            0.817
  Correlation   0.905
=============== Average Calibration Metrics ================
  Root-mean-squared Calibration Error   0.075
  Mean-absolute Calibration Error       0.053
  Miscalibration Area                   0.054
========== Adversarial Group Calibration Metrics ===========
  Mean-absolute Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.129
     Group Size: 0.56 -- Calibration Error: 0.077
     Group Size: 1.00 -- Calibration Error: 0.053
  Root-mean-squared Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.143
     Group Size: 0.56 -- Calibration Error: 0.100
     Group Size: 1.00 -- Calibration Error: 0.075
==============

In [11]:
maes

array([0.3958398 , 0.35454262, 0.29128523, 0.32083054, 0.36778862])

In [12]:
maes.mean()

0.34605736223726014

In [13]:
uncertainties

array([0.2193156 , 0.19080055, 0.17977706, 0.18877757, 0.17967668])

In [14]:
uncertainties.mean()

0.19166949093341829

In [15]:
metricss

[{'accuracy': {'mae': 0.39583979969546573,
   'rmse': 0.8585004758684266,
   'mdae': 0.07177907228469849,
   'marpd': 134.32681348338014,
   'r2': 0.7144733248240083,
   'corr': 0.8640669855670066},
  'avg_calibration': {'rms_cal': 0.07052779714067306,
   'ma_cal': 0.05581622678396872,
   'miscal_area': 0.056357731128329994},
  'adv_group_calibration': {'ma_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
    'adv_group_cali_mean': array([0.42358081, 0.12136151, 0.09064195, 0.07555024, 0.07019225,
           0.07075613, 0.06661291, 0.06351702, 0.06030773, 0.05581623]),
    'adv_group_cali_stderr': array([4.05586300e-02, 3.27459087e-02, 1.67073727e-02, 7.47382366e-03,
           5.53433886e-03, 6.08288658e-03, 3.61350826e-03, 1.79924678e-03,
           1.86031052e-03, 7.31423639e-18])},
   'rms_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.2222222

#### Conclusion

This value of the MAE will be our benchmark value for comparing the different methods in this repository.